###### RDKit is collection of cheminformatics and machine learning sofware written in c++ and python. 

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [2]:
#open the data we saved in last part(data collection)
df=pd.read_csv('D:\TPKR_data2.csv')
df

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity
0,CHEMBL330863,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,128.0,active
1,CHEMBL124660,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,220.0,active
2,CHEMBL126699,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,8790.0,intermediate
3,CHEMBL445636,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,1910.0,intermediate
4,CHEMBL941,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,30000.0,inactive
...,...,...,...,...
3457,CHEMBL4289301,CCC[C@H](C)Nc1ncc2c(-c3ccnc(N4CCNCC4)c3)cn([C@...,43.0,NaN
3458,CHEMBL4284846,CCC[C@H](C)Nc1ncc2c(-c3ccnc(C(=O)OC)c3)cn([C@H...,558.0,NaN
3459,CHEMBL4292717,CCC[C@H](C)Nc1ncc2c(-c3cnc(C4CC4)nc3)cn([C@H]3...,13790.0,NaN
3460,CHEMBL4293116,CCC[C@H](C)Nc1ncc2c(-c3cnn(C4CCNCC4)c3)cn([C@H...,45.0,NaN


###### Let's remove NaN value from bioactivity and canonical smiles column 

In [44]:
df1=df[df.bioactivity.notna()]
df2=df1[df1.canonical_smiles.notna()]
df2

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity
0,CHEMBL330863,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,128.0,active
1,CHEMBL124660,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,220.0,active
2,CHEMBL126699,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,8790.0,intermediate
3,CHEMBL445636,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,1910.0,intermediate
4,CHEMBL941,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,30000.0,inactive
...,...,...,...,...
3378,CHEMBL3809489,Nc1nc(Nc2ccc3c(c2)CC[C@@H](N2CCCC2)CC3)nn1-c1c...,0.9,active
3379,CHEMBL4207004,CNc1ncc2c(-c3ccc(N4CCN(C(=O)C(C)(C)C)CC4)nc3)n...,1200.0,active
3380,CHEMBL4217649,CCN1CCN(c2ccc(-c3nn(Cc4cn(Cc5ccccc5)nn4)c4nc(N...,10000.0,inactive
3381,CHEMBL4218369,CNc1ncc2c(-c3ccc(N4CCN(C)CC4)nc3)nn(Cc3cn(Cc4c...,700.0,active



### "Lipinski Rule of Five"

Lipinski rule of 5 helps in distinguishing between drug like and non drug like molecules. It predicts high probability of success or failure due to drug likeness for molecules complying with 2 or more of the following rules

Molecular mass less than 500 Dalton
High lipophilicity (expressed as LogP less than 5)
Less than 5 hydrogen bond donors
Less than 10 hydrogen bond acceptors
Molar refractivity should be between 40-130

lets define a function Lipinski to calculate molecular weight, logP, number of H donor and number of H acceptor.


In [46]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

#####  Apply this function on canonical_smiles column in order to get values for various molecules present in our dataset

In [48]:
df_lipinski = lipinski(df2.canonical_smiles)
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,576.742,5.28050,1.0,8.0
1,562.715,5.03450,1.0,8.0
2,543.672,4.50748,1.0,8.0
3,543.672,4.36498,1.0,8.0
4,493.615,4.59032,2.0,7.0
...,...,...,...,...
3299,506.658,4.88200,2.0,8.0
3300,565.686,3.31280,1.0,11.0
3301,509.622,2.76000,1.0,11.0
3302,495.595,2.36990,1.0,11.0


In [49]:
#combine two dataframes to create a new dataset
df_combined = pd.concat([df2,df_lipinski], axis=1)

In [50]:
df_combined

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL330863,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,128.0,active,576.742,5.28050,1.0,8.0
1,CHEMBL124660,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,220.0,active,562.715,5.03450,1.0,8.0
2,CHEMBL126699,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,8790.0,intermediate,543.672,4.50748,1.0,8.0
3,CHEMBL445636,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,1910.0,intermediate,543.672,4.36498,1.0,8.0
4,CHEMBL941,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,30000.0,inactive,493.615,4.59032,2.0,7.0
...,...,...,...,...,...,...,...,...
3378,CHEMBL3809489,Nc1nc(Nc2ccc3c(c2)CC[C@@H](N2CCCC2)CC3)nn1-c1c...,0.9,active,NaN,NaN,NaN,NaN
3379,CHEMBL4207004,CNc1ncc2c(-c3ccc(N4CCN(C(=O)C(C)(C)C)CC4)nc3)n...,1200.0,active,NaN,NaN,NaN,NaN
3380,CHEMBL4217649,CCN1CCN(c2ccc(-c3nn(Cc4cn(Cc5ccccc5)nn4)c4nc(N...,10000.0,inactive,NaN,NaN,NaN,NaN
3381,CHEMBL4218369,CNc1ncc2c(-c3ccc(N4CCN(C)CC4)nc3)nn(Cc3cn(Cc4c...,700.0,active,NaN,NaN,NaN,NaN


In [51]:
#remove NaN value from Molecular weight column
df3=df_combined[df_combined.MW.notna()]
df3

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL330863,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,128.0,active,576.742,5.28050,1.0,8.0
1,CHEMBL124660,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,220.0,active,562.715,5.03450,1.0,8.0
2,CHEMBL126699,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,8790.0,intermediate,543.672,4.50748,1.0,8.0
3,CHEMBL445636,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,1910.0,intermediate,543.672,4.36498,1.0,8.0
4,CHEMBL941,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,30000.0,inactive,493.615,4.59032,2.0,7.0
...,...,...,...,...,...,...,...,...
3299,CHEMBL4218371,CC(C)n1cnc2c(Nc3ccc(Br)cc3)nc(N[C@H]3CC[C@H](N...,10.0,active,506.658,4.88200,2.0,8.0
3300,CHEMBL4205021,CC(O)CNc1nc(Nc2ccc(Br)cc2)c2ncn(C(C)C)c2n1,591.0,intermediate,565.686,3.31280,1.0,11.0
3301,CHEMBL4216401,N[C@H]1CC[C@H](Nc2nc(Nc3ccc(Br)cc3)c3ncn(C4CCC...,4.0,inactive,509.622,2.76000,1.0,11.0
3302,CHEMBL4207535,N[C@H]1CC[C@H](Nc2nc(Nc3ccc(Cl)cc3)c3ncn(C4CCC...,5.0,active,495.595,2.36990,1.0,11.0



The nature of potency values is logarithmic

If you look at dose-response curves, they are sigmoidal when you plot them in logarithmic space.

Using pIC50 is the proper way to think about the data.

If your potency goes down because you've gone from micromolar to nanomolar, that’s an exponential change, not a linear change.

pIC50 is really the right way to think about potency of compounds

lets create a function(pIC50) that converts potency to log value  

In [52]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x


In [56]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [57]:
df_norm = norm_value(df3)
df_norm

C:\Users\Jash trivedi\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,molecule_chembl_id,canonical_smiles,bioactivity,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,CHEMBL330863,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,active,576.742,5.28050,1.0,8.0,128.0
1,CHEMBL124660,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,active,562.715,5.03450,1.0,8.0,220.0
2,CHEMBL126699,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,intermediate,543.672,4.50748,1.0,8.0,8790.0
3,CHEMBL445636,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,intermediate,543.672,4.36498,1.0,8.0,1910.0
4,CHEMBL941,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,inactive,493.615,4.59032,2.0,7.0,30000.0
...,...,...,...,...,...,...,...,...
3299,CHEMBL4218371,CC(C)n1cnc2c(Nc3ccc(Br)cc3)nc(N[C@H]3CC[C@H](N...,active,506.658,4.88200,2.0,8.0,10.0
3300,CHEMBL4205021,CC(O)CNc1nc(Nc2ccc(Br)cc2)c2ncn(C(C)C)c2n1,intermediate,565.686,3.31280,1.0,11.0,591.0
3301,CHEMBL4216401,N[C@H]1CC[C@H](Nc2nc(Nc3ccc(Br)cc3)c3ncn(C4CCC...,inactive,509.622,2.76000,1.0,11.0,4.0
3302,CHEMBL4207535,N[C@H]1CC[C@H](Nc2nc(Nc3ccc(Cl)cc3)c3ncn(C4CCC...,active,495.595,2.36990,1.0,11.0,5.0


In [58]:
df_norm.describe()

,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
count,3304.000000,3304.000000,3304.000000,3304.000000,3229.000000
mean,455.619918,4.132873,1.667373,7.098971,5577.870033
std,79.696724,1.097207,1.239847,2.063526,9313.582574
min,210.236000,-1.438100,0.000000,1.000000,0.101000
25%,413.452250,3.459190,1.000000,6.000000,42.380000
50%,461.614000,4.197210,2.000000,8.000000,2216.000000
75%,506.566000,4.870500,2.000000,9.000000,10000.000000
max,1412.367000,10.443700,11.000000,15.000000,322000.000000


In [61]:
df_norm.standard_value_norm.describe()

count      3229.000000
mean       5577.870033
std        9313.582574
min           0.101000
25%          42.380000
50%        2216.000000
75%       10000.000000
max      322000.000000
Name: standard_value_norm, dtype: float64

In [62]:
#converting IC50 value to pIC50 value
df_final = pIC50(df_norm)
df_final

,molecule_chembl_id,canonical_smiles,bioactivity,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL330863,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,active,576.742,5.28050,1.0,8.0,6.892790
1,CHEMBL124660,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,active,562.715,5.03450,1.0,8.0,6.657577
2,CHEMBL126699,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,intermediate,543.672,4.50748,1.0,8.0,5.056011
3,CHEMBL445636,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,intermediate,543.672,4.36498,1.0,8.0,5.718967
4,CHEMBL941,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,inactive,493.615,4.59032,2.0,7.0,4.522879
...,...,...,...,...,...,...,...,...
3299,CHEMBL4218371,CC(C)n1cnc2c(Nc3ccc(Br)cc3)nc(N[C@H]3CC[C@H](N...,active,506.658,4.88200,2.0,8.0,8.000000
3300,CHEMBL4205021,CC(O)CNc1nc(Nc2ccc(Br)cc2)c2ncn(C(C)C)c2n1,intermediate,565.686,3.31280,1.0,11.0,6.228413
3301,CHEMBL4216401,N[C@H]1CC[C@H](Nc2nc(Nc3ccc(Br)cc3)c3ncn(C4CCC...,inactive,509.622,2.76000,1.0,11.0,8.397940
3302,CHEMBL4207535,N[C@H]1CC[C@H](Nc2nc(Nc3ccc(Cl)cc3)c3ncn(C4CCC...,active,495.595,2.36990,1.0,11.0,8.301030


In [64]:
df_final.pIC50.describe()

count    3229.000000
mean        6.208840
std         1.380527
min         3.492144
25%         5.000000
50%         5.654430
75%         7.372839
max         9.995679
Name: pIC50, dtype: float64

In [69]:
#Remove the column which shows intermediate bioactivity 
df_new = df_final[df_final['bioactivity'] != 'intermediate']
df_new


,molecule_chembl_id,canonical_smiles,bioactivity,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL330863,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,active,576.742,5.28050,1.0,8.0,6.892790
1,CHEMBL124660,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,active,562.715,5.03450,1.0,8.0,6.657577
4,CHEMBL941,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,inactive,493.615,4.59032,2.0,7.0,4.522879
5,CHEMBL124035,COCCOc1cc2ncnc(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC...,active,495.580,3.80490,1.0,8.0,6.346787
6,CHEMBL125898,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,active,564.687,3.88080,1.0,9.0,7.397940
...,...,...,...,...,...,...,...,...
3298,CHEMBL4218162,Nc1ncnc2c1c(-c1ccc(Oc3ccccc3)cc1)cn2[C@H]1CC[C...,active,405.550,3.47700,1.0,7.0,8.301030
3299,CHEMBL4218371,CC(C)n1cnc2c(Nc3ccc(Br)cc3)nc(N[C@H]3CC[C@H](N...,active,506.658,4.88200,2.0,8.0,8.000000
3301,CHEMBL4216401,N[C@H]1CC[C@H](Nc2nc(Nc3ccc(Br)cc3)c3ncn(C4CCC...,inactive,509.622,2.76000,1.0,11.0,8.397940
3302,CHEMBL4207535,N[C@H]1CC[C@H](Nc2nc(Nc3ccc(Cl)cc3)c3ncn(C4CCC...,active,495.595,2.36990,1.0,11.0,8.301030


In [70]:
#check how many active and inactive molecules we have in our dataset
df_new.bioactivity.value_counts()

active      1517
inactive    1277
Name: bioactivity, dtype: int64

In [71]:
df_new.to_csv('data_eda.csv')